### Challenge Instructions

Your client is a broker for orthopedic surgeries who wants to understand the performance of their surgeons. He is asking you to analyze his data and answer the following question:

##### Who are the most and least skilful surgeons for hip replacement operations?

Make sure to include all your work in your email when sending us your answers. You are welcome to use tools of your choice when completing this task, so long as you send all relevant documents together with your answer (for example, include the SQL queries if you are using SQL to analyze the data). Be sure to carefully study the background information.


#### Background Information
● The client works with data from the EQ-5D-5L questionnaire - a survey that is taken by patients both before and after their operations to measure outcome success. By comparing the scores, the improvement in how patients feel in various areas of their lives can be evaluated.
● The questionnaire consists of five sections: mobility, self-care, usual activities, pain/discomfort, and anxiety/depression.
● The success of the surgery is evaluated by comparing the quality of life pre and post-operation.
● The answers will then be converted to a health state score. Please see below for an example questionnaire and an example of conversion.
● Based on the example questionnaire (on the next page), Patient A has reported their symptoms to be the following:
    ○ Mobility: Slight problems (“Slight”)
    ○ Self-care: Moderate problems (“Moderate”)
    ○ Usual activities: Slight problems (“Slight”)
    ○ Pain/discomfort: Severe problems (“Severe”)
    ○ Anxiety/depression: Extreme problems (“Unable”)
● The value set you will need for the conversion from individual answers to a health state score is part of the data set, in the table “answer_options”. Each answer has a corresponding “central estimate”. The “None” category is always 0, therefore it was omitted from the table.
● In order to receive a score for the patient’s health state, it is necessary to deduct the sum of the individual scores from 1, as can be seen in the table above. Therefore, a higher score means better health. In our example, the overall health score would be 0.247, calculated as follows:

1 - (0.058 + 0.080 + 0.050 + 0.276 + 0.289) = 0.247

### Data Understanding

In [ ]:
answer_options = _deepnote_execute_sql('SELECT * FROM answer_options ', 'SQL_E13E9A73_6ADB_4B5F_99BE_A16662286AD7', audit_sql_comment='', sql_cache_mode='cache_disabled')
answer_options

,question_id,answer,severity_code,central_estimate
0,1,I have no problems in walking around,1,0.000
1,1,I have slight problems in walking around,2,0.058
2,1,I have moderate problems in walking around,3,0.076
3,1,I have severe problems in walking around,4,0.207
4,1,I am unable to walk around,5,0.274
5,2,I have no problems washing or dressing myself,1,0.000
6,2,I have slight problems washing or dressing myself,2,0.050
7,2,I have moderate problems washing or dressing m...,3,0.080
8,2,I have severe problems washing or dressing myself,4,0.164
9,2,I am unable to wash or dress myself,5,0.203


In [ ]:
questions = _deepnote_execute_sql('SELECT * FROM questions ', 'SQL_E13E9A73_6ADB_4B5F_99BE_A16662286AD7', audit_sql_comment='', sql_cache_mode='cache_disabled')
questions

,id,title,description
0,1,Mobility,Please indicate what applies
1,2,Self-care,Please indicate what applies
2,3,Usual activities,Please indicate what applies
3,4,Pain / discomfort,Please indicate what applies
4,5,Anxiety / depression,Please indicate what applies


In [ ]:
questionnaires = _deepnote_execute_sql('SELECT * FROM questionnaires', 'SQL_E13E9A73_6ADB_4B5F_99BE_A16662286AD7', audit_sql_comment='', sql_cache_mode='cache_disabled')
questionnaires

,id,type,treatment,questions
0,1,pre,Hip,"[1, 2, 3, 4, 5]"
1,2,post,Hip,"[1, 2, 3, 4, 5]"
2,3,pre,Knee,"[1, 2, 3, 4, 5]"
3,4,post,Knee,"[1, 2, 3, 4, 5]"


In [ ]:
answers = _deepnote_execute_sql('SELECT * FROM answers', 'SQL_E13E9A73_6ADB_4B5F_99BE_A16662286AD7', audit_sql_comment='', sql_cache_mode='cache_disabled')
answers

,id,question_id,patient_id,questionnaire_id,answer
0,1,1,1,1,I am unable to walk around
1,2,2,1,1,I have slight problems washing or dressing myself
2,3,3,1,1,I have no problems doing my usual activities
3,4,4,1,1,I have severe pain or discomfort
4,5,5,1,1,I am not anxious or depressed
...,...,...,...,...,...
219075,219076,1,25000,2,I am unable to walk around
219076,219077,2,25000,2,I have severe problems washing or dressing myself
219077,219078,3,25000,2,I have slight problems doing my usual activities
219078,219079,4,25000,2,I have moderate pain or discomfort


In [ ]:
surgeons = _deepnote_execute_sql('SELECT * FROM surgeons', 'SQL_E13E9A73_6ADB_4B5F_99BE_A16662286AD7', audit_sql_comment='', sql_cache_mode='cache_disabled')
surgeons

,id,name
0,1,Princess Leia
1,2,Boba Fett
2,3,Darth Vader
3,4,Han Solo
4,5,Obi-Wan Kenobi
5,6,Padme Amidala
6,7,Darth Maul
7,8,Mon Mothma
8,9,Darth Sidious
9,10,Luke Skywalker


In [ ]:
patients = _deepnote_execute_sql('SELECT * FROM patients', 'SQL_E13E9A73_6ADB_4B5F_99BE_A16662286AD7', audit_sql_comment='', sql_cache_mode='cache_disabled')
patients

,id,gender,surgeon_id
0,1,Male,8
1,2,Male,10
2,3,Female,7
3,4,Male,6
4,5,Female,6
...,...,...,...
24995,24996,Female,6
24996,24997,Male,3
24997,24998,Female,1
24998,24999,Male,10


### Data preparation

In [ ]:
answer_reduc = _deepnote_execute_sql('-- Dataset reduction\n\nWITH base AS (\n    SELECT *\n        , CASE WHEN questionnaire_id = 1 THEN \'pre\' ELSE \'post\' END AS type\n\n-- Check if there is data for pre and post\n        , MAX(questionnaire_id) OVER(PARTITION BY patient_id) AS max_id\n        , MIN(questionnaire_id) OVER(PARTITION BY patient_id) AS min_id\n    FROM answers \n\n-- Remove the Knee treatments\n    WHERE questionnaire_id in (1,2)\n)\n\n-- Select only patients that have pre and post results\n, pre_post AS (\n    SELECT id\n    , question_id\n    , patient_id\n    , answer\n    , type\nFROM base \nWHERE max_id = 2 AND min_id = 1\n)\n\nSELECT \n    pre_post.*\n    , code.severity_code\n    , code.central_estimate\nFROM pre_post\nLEFT JOIN answer_options AS code USING(question_id, answer)', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
answer_reduc

,id,question_id,patient_id,answer,type,severity_code,central_estimate
0,222,2,26,I have no problems washing or dressing myself,pre,1,0.000
1,223,3,26,I have severe problems doing my usual activities,pre,4,0.162
2,224,4,26,I have moderate pain or discomfort,pre,3,0.084
3,225,5,26,I am severely anxious or depressed,pre,4,0.285
4,226,1,26,I have no problems in walking around,post,1,0.000
...,...,...,...,...,...,...,...
96925,214626,1,24488,I have moderate problems in walking around,post,3,0.076
96926,214621,1,24488,I have moderate problems in walking around,pre,3,0.076
96927,218561,1,24939,I have moderate problems in walking around,post,3,0.076
96928,218861,1,24976,I have moderate problems in walking around,pre,3,0.076


In [ ]:
health_state = _deepnote_execute_sql('-- Scores calculation\n\nSELECT \n    name AS surgeon_name\n    , patient_id\n    , 1 - SUM(CASE WHEN type = \'pre\' THEN central_estimate END) AS \'pre\'\n    , 1 - SUM(CASE WHEN type = \'post\' THEN central_estimate END) AS \'post\'\n    , post - pre as score\nFROM answer_reduc  AS h\nLEFT JOIN patients AS p ON h.patient_id = p.id\nLEFT JOIN surgeons AS s ON s.id = p.surgeon_id\n\nGROUP BY 1,2', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
health_state

,surgeon_name,patient_id,pre,post,score
0,Padme Amidala,67,0.382,0.858,0.476
1,Luke Skywalker,118,0.391,0.702,0.311
2,Darth Vader,134,0.388,0.709,0.321
3,Boba Fett,280,0.449,0.924,0.475
4,Padme Amidala,301,0.510,0.678,0.168
...,...,...,...,...,...
9688,Mon Mothma,7458,0.212,0.596,0.384
9689,Darth Maul,14341,0.453,0.788,0.335
9690,Princess Leia,8548,0.350,0.736,0.386
9691,Darth Maul,11885,0.528,0.809,0.281


<hr>

In [ ]:
# Group the data by surgeon and calculate the average score difference and standard deviation 

surgeon_performance = health_state.groupby('surgeon_name').score.agg(
    avg_score='mean',
    std_score='std',
    min_score='min',
    max_score='max'
).reset_index()

surgeon_performance['negative_improvement_rate'] = health_state.groupby('surgeon_name').apply(
    lambda x: (x['score'] < 0).mean()
).values

surgeon_performance = surgeon_performance.sort_values(
    by=['avg_score', 'std_score', 'negative_improvement_rate'],
    ascending=[False, True, True]
).reset_index(drop=True)

In [ ]:
import plotly.express as px

# Prepare the data specifically for the box plot
box_plot_data = health_state.reset_index()

# Create the box plot
fig = px.box(box_plot_data, x='score', y='surgeon_name',
            labels={'score_change': 'Change in Health State Score'},
            notched=True,
            title='Distribution of Health State Score Changes by Surgeon')

# Show the figure
fig.show()

<hr>

### Who are the most and least skillful surgeons for hip replacement operations?

##### In a glance: The global most skillful surgeons for hip replacement are Padme Amidala, Obi-Wan Kenobi, and Darth Sidious. Conversely, the least skillful surgeons are Luke Skywalker, Darth Vader, and Mon Mothma.

To refine the evaluation, some criteria are considered to reflect not only improving outcomes on average but also delivering consistent and safer results.


**Average Score:**

> Top Surgeons:
> 1- Padme Amidala with an average score of 0.132
> 2- Obi-Wan Kenobi with an average score of 0.131
> 3- Darth Sidious with an average score of 0.124
>
> Bottom Surgeons: 
> 1- Luke Skywalker with an average score of 0.111
> 2- Darth Vader with an average score of 0.112
> 3- Mon Mothma with an average score of 0.113
 
Notably, Padme Amidala exhibits not only a high average score but also a low negative improvement rate and the highest maximum score, signifying consistent excellence. 
Conversely, Luke Skywalker's low average score, coupled with the highest negative improvement rate and lowest maximum score, suggests consistent underperformance.

**Standard Deviation of Scores (std_score):** 

Lower standard deviations indicate greater consistency in performance. Surgeons like Mon Mothma and Darth Vader exhibit the least variability, indicating a more predictable performance pattern. Conversely, higher-ranked surgeons such as Padme Amidala and Obi-Wan Kenobi display higher deviations, implying a broader spectrum of outcomes ranging from peaks to troughs.

> Top Surgeons (Lowest rate):
> 1- Mon Mothma with a standard deviation of 0.251
> 2- Darth Vader with a standard deviation of 0.252
> 3- Princess Leia with a standard deviation of 0.252
>
> Bottom Surgeons (Highest rate):
> 1- Luke Skywalker with a standard deviation of 0.264
> 2- Padme Amidala with a standard deviation of 0.261
> 3- Obi-Wan Kenobi with a standard deviation of 0.260


**Negative Improvement Rate:**

Measures the frequency of performance declines. Obi-Wan Kenobi has the lowest rate, suggesting fewer instances where performance worsened. The same three surgeons that are in the top three of the average score are also the ones with the lowest negative improvement rates.

> Top Surgeons (Lowest rate):
> 1- Obi-Wan Kenobi with a negative improvement rate of 0.297
> 2- Padme Amidala with a negative improvement rate of 0.301
> 3- Darth Sidious with a negative improvement rate of 0.305
> 
> Bottom Surgeons (Highest rate):
> 1- Luke Skywalker with a negative improvement rate of 0.342
> 2- Princess Leia with a negative improvement rate of 0.334
> 3- Darth Maul with a negative improvement rate of 0.325


**Conclusion:**
The analysis underscores the importance of considering both average outcomes and consistency when evaluating surgeons' performance. While high average scores indicate proficiency, factors such as standard deviation and negative improvement rates provide valuable insights into reliability and safety. Surgeons like Padme Amidala and Obi-Wan Kenobi emerge as exemplary performers, showcasing not only superior skill but also consistent and reliable outcomes. Conversely, surgeons like Luke Skywalker exhibit consistent underperformance, highlighting the significance of addressing variability and enhancing skill proficiency for improved surgical outcomes.


For a clearer visual representation, refer to the chart below. Here, the highest points correspond to higher average scores, while points horizontally closer from the origin indicate a lower rate of performance declines. Ideally, the top-performing surgeons should cluster on the top, near the y-axis for optimal positioning.

In [ ]:
DeepnoteChart(surgeon_performance, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"circle","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"negative_improvement_rate","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"quantitative","field":"avg_score","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"nominal","field":"surgeon_name","scale":{"scheme":"paired"}}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"tooltipDefaultMode":true}}""")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Add traces for min, max, and mean scores
fig.add_trace(go.Scatter(
    x=surgeon_performance['surgeon_name'], y=surgeon_performance['min_score'],
    mode='markers', name='Min Score',
    marker=dict(color='lightsalmon', size=8)
))
fig.add_trace(go.Scatter(
    x=surgeon_performance['surgeon_name'], y=surgeon_performance['max_score'],
    mode='markers', name='Max Score',
    marker=dict(color='lightgreen', size=8)
))
fig.add_trace(go.Scatter(
    x=surgeon_performance['surgeon_name'], y=surgeon_performance['avg_score'],
    mode='markers', name='Average Score',
    marker=dict(color='skyblue', size=8)
))

fig.update_layout(
    title='Min, Max, and Average Health State Score Changes by Surgeon',
    xaxis_title='Surgeon Name',
    yaxis_title='Score Change',
    legend_title='Score Type',
    # plot_bgcolor='white'
)

fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2254e57f-f4d0-4158-b195-020957cbda5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>